In [1]:
import git
g = git.cmd.Git('/root/Projects/Assignment-Projects')
g.pull()

'Already up-to-date.'

In [1]:
from Sparker.SparkLogProcesser.SparkLogFileHandler import setSparkContext
from pyspark import SparkContext
sc = SparkContext() 
setSparkContext(sc)

In [2]:
from Sparker.Logic.LogicTests import *
day2Logs = readParsedLogsFromHDFS(entireDayParsedLogsFolder2)

hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/Parsed/2016-12-26_parsed has been read by 2017-05-09 15:04:22.658010


In [3]:
from MainSrc.PythonVersionHandler import *
from paths import *

In [4]:
from Sparker.Logic.LogicTests import *
from Sparker.Logic.Trainer import *
#trainPairWiseData(data) 

In [5]:
def printSeparater(c = 3):
    for n in range(c):
        print_('#' * 88)

In [6]:
from sklearn import svm

def evaluateModelOnData(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict([p.features])[0]))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseData(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    model = svm.SVC()#kernel =  'poly', degree = 3) #SVMWithSGD.train(data, iterations=100)
    X =  data.map(lambda p: p.features).collect()
    Y =  data.map(lambda p: p.label).collect()
    model.fit(X, Y)
    #print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())
    #print_('The learned weights:\n' + str(model.coef_).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnData(model, data, dataName, modelName)
    return model

def runTrainingExperiment(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    trainData = scaleTrainData(trainData)
    testData = scaleTrainData(testData)
    model = trainPairWiseData(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnData(model, testData, 'testData', modelName)

In [7]:
def featureFilter(trainData):
    return trainData.map(lambda x: LabeledPoint(x.label, list(x.features[1:4]) + list(x.features[5:6]) + list(x.features[7:-2])))

def runExperiment(trainData, modelName = 'Model'):
    printSeparater(1)
    testData, trainData = trainData.randomSplit(weights=[0.3, 0.7])
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
    print_('The experiment for', modelName, 'has been completed by', nowStr())
    printSeparater(1)
    
def trainPairWiseDataTestKeyword2(keyword):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    outputFolder = joinPath(HDFSDataFolder, 'Day1_' + keyword + '_Data')
    trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_TrainData')
    trainData = readTrainDataFromHDFS(trainDataFile)
    runExperiment(trainData, modelName = 'ExtendedModel')
    trainData = featureFilter(trainData)
    runExperiment(trainData, modelName = 'FilteredModel')
    printSeparater(2)

In [8]:
#trainPairWiseDataTestKeyword2('iphone 6')

In [13]:
import pickle
def savePairWisetrainData(keyword):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    outputFolder = joinPath(HDFSDataFolder, 'Day1_' + keyword + '_Data')
    dataTypes = ['_TrainData', '_labeledPairs', '_journey_products']
    for typ in dataTypes:
        fileName = inputName + '_' + keyword + typ
        trainDataFile = joinPath(outputFolder, fileName)
        trainData = sc_().textFile(trainDataFile)
        f = joinPath('/root/Projects/TrainData6', fileName)
        trainData.coalesce(1).saveAsTextFile(f)
        print_(f, 'has been saved successfully by', nowStr())
        continue
        trainData = trainData.collect()
        fp = open(f, "wb")   #Pickling
        pickle.dump(trainData, fp)

In [10]:
def trainPairWiseDataTest():
    keywords = ['iphone 6', 'jant', 'nike air max', 'tv unitesi', 'kot ceket', 'camasir makinesi', 'bosch']
    for keyword in keywords[3:4]:
        #trainPairWiseDataTestKeyword2(keyword)readTrainDataFromHDFS
        savePairWisetrainData(keyword)

In [11]:
def main():
    printSeparater()
    print_('%s:' % nowStr(), 'Running on osldevptst02:Jupyter...')

    trainPairWiseDataTest()

    print_('%s:' % nowStr(), 'DONE')
    printSeparater()


In [15]:
main()

########################################################################################
########################################################################################
########################################################################################
2017-05-09 15:09:16.404133: Running on osldevptst02:Jupyter...
########################################################################################
########################################################################################
/root/Projects/TrainData6/all_day_tv_unitesi_TrainData has been saved successfully by 2017-05-09 15:09:35.835378
/root/Projects/TrainData6/all_day_tv_unitesi_labeledPairs has been saved successfully by 2017-05-09 15:09:56.224687
/root/Projects/TrainData6/all_day_tv_unitesi_journey_products has been saved successfully by 2017-05-09 15:09:58.585784
2017-05-09 15:09:58.589724: DONE
########################################################################################
#####################